In [1]:
print('merge_notifica')
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica, auto_fit_columns
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","merge_notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

merge_notifica


In [2]:
cc = CasosConfirmados()
cc.load()
print(cc.shape())

notifica = Notifica()
notifica.load()

print(notifica.shape())

(661109, 11982, 472175, 176952)
(2542603, 1156088, 12736, 1344092, 2322)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

(2542603, 41)


In [4]:
idx_casos_confirmados = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

(27948, 25)

In [5]:
idx_casos_notifica = casosn.loc[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn.loc[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn.loc[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.shape

(743199, 41)

In [6]:
#merge hash_resid
casos_hash_resid = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])]
print(f"casos_hash_resid {len(casos_hash_resid)}")
notifica_hash_resid = casosn.loc[(casosn['hash_resid'].isin(casosc['hash_resid']))]
print(f"notifica_hash_resid {len(notifica_hash_resid)}")
merge_hash_resid = pd.merge(casos_hash_resid,notifica_hash_resid[['hash_resid','id']])
print(f"merge_hash_resid {len(merge_hash_resid)}")
merge_hash_resid = merge_hash_resid.drop_duplicates('ordem')
print(f"merge_hash_resid {len(merge_hash_resid)}\n")

#merge hash_atend
casos_hash_atend = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])]
print(f"casos_hash_atend {len(casos_hash_atend)}")
notifica_hash_atend = casosn.loc[(casosn['hash_atend'].isin(casosc['hash_atend']))]
print(f"notifica_hash_atend {len(notifica_hash_atend)}")
merge_hash_atend = pd.merge(casos_hash_atend,notifica_hash_atend[['hash_atend','id']])
print(f"merge_hash_atend {len(merge_hash_atend)}")
merge_hash_atend = merge_hash_atend.drop_duplicates('ordem')
print(f"merge_hash_atend {len(merge_hash_atend)}\n")

#merge hash_diag
casos_hash_diag = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])]
print(f"casos_hash_diag {len(casos_hash_diag)}")
notifica_hash_diag = casosn.loc[(casosn['hash_diag'].isin(casosc['hash_diag']))]
print(f"notifica_hash_diag {len(notifica_hash_diag)}")
merge_hash_diag = pd.merge(casos_hash_diag,notifica_hash_diag[['hash_diag','id']])
print(f"merge_hash_diag {len(merge_hash_diag)}")
merge_hash_diag = merge_hash_diag.drop_duplicates('ordem')
print(f"merge_hash_diag {len(merge_hash_diag)}\n")

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(casos_hash_resid.index.tolist() + casos_hash_atend.index.tolist() + casos_hash_diag.index.tolist())
print(f"idx_casos_confirmados {len(idx_casos_confirmados)}")

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
print(f"casos_confirmados_nao_notifica {len(casos_confirmados_nao_notifica)}") ## aqui o numero total é 27910 casos não encontrados no notifica
len(idx_casos_confirmados) + len(casos_confirmados_nao_notifica)

casos_hash_resid 609828
notifica_hash_resid 721136
merge_hash_resid 721136
merge_hash_resid 609828

casos_hash_atend 577355
notifica_hash_atend 663534
merge_hash_atend 670342
merge_hash_atend 577355

casos_hash_diag 554154
notifica_hash_diag 557416
merge_hash_diag 563991
merge_hash_diag 554154

idx_casos_confirmados 633161
casos_confirmados_nao_notifica 27948


661109

In [7]:
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).sort_values('ordem')
ordem_duplicados = merge_casos_notifica.loc[merge_casos_notifica.duplicated('ordem', keep=False)]
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).drop_duplicates('ordem')

print(len(merge_casos_notifica))

661109


In [8]:
casos_merged = merge_casos_notifica.copy()
casos_merged['id'] = casos_merged['id'].fillna('-1').astype(int)

casos_merged.loc[casos_merged['recuperado']==1,'evolucao'] = 1
casos_merged.loc[casos_merged['obito']==1,'evolucao'] = 2
casos_merged.loc[casos_merged['ativo']==1,'evolucao'] = 3

casos_merged = casos_merged.rename(columns={
    'identificacao': 'id_access',
    'laboratorio': 'exame',
    'data_obito': 'data_evolucao',
    'data_com_obito': 'data_com_evolucao'
    })

casos_merged.loc[(casos_merged['data_com_evolucao'].isnull() & (casos_merged['evolucao']==2)),'data_com_evolucao'] = casos_merged.loc[(casos_merged['data_com_evolucao'].isnull() & (casos_merged['evolucao']==2)),'data_evolucao']

casos_merged = casos_merged[['id_access','id','paciente','sexo','idade','rs','ibge_resid','mun_resid','data_1o_sintomas','ibge_atend','mun_atend','exame','data_diagnostico','data_comunicacao','evolucao','data_evolucao','data_com_evolucao']].sort_values('id_access')

obitos_merged = casos_merged.loc[casos_merged['evolucao']==2,['id_access','id','paciente','sexo','idade','rs','ibge_resid','mun_resid','data_diagnostico','data_evolucao','data_com_evolucao']].sort_values('data_com_evolucao')

print(casos_merged.shape)
print(obitos_merged.shape)

,id_access,id,paciente,sexo,idade,rs,ibge_resid,mun_resid,data_1o_sintomas,ibge_atend,mun_atend,exame,data_diagnostico,data_comunicacao,evolucao,data_evolucao,data_com_evolucao
641,1,111834,ABBAS MOHAMAD MESSELMANI,M,41,9,4108304,FOZ DO IGUACU,2020-04-11,4108304,FOZ DO IGUACU,LACEN,2020-04-17,2020-04-18,1.0,NaT,NaT
24648,2,287400,ABDIEL DE LIMA GONCALVES,M,44,14,4117107,NOVA LONDRINA,2020-06-22,4117107,NOVA LONDRINA,IBMP,2020-07-03,2020-07-04,1.0,NaT,NaT
5513,3,-1,ABDOU INDIAYE,M,31,20,4127700,TOLEDO,2020-05-29,4127700,TOLEDO,IBMP,2020-06-03,2020-06-04,1.0,NaT,NaT
5846,4,176206,ABEDIEL DA SILVA,M,38,3,4112009,JAGUARIAIVA,NaT,4112009,JAGUARIAIVA,COVID-19 IGG/IGM ECO TESTE,2020-06-05,2020-06-06,1.0,NaT,NaT
9420,5,203117,ABEGAIL DOMINGUES,F,59,2,4125506,SAO JOSE DOS PINHAIS,2020-06-09,4125506,SAO JOSE DOS PINHAIS,IBMP,2020-06-16,2020-06-17,1.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721130,742941,2570007,ZOLEIDE MARIA COLODA ZANELLA,F,59,10,4104808,CASCAVEL,2021-02-26,4104808,CASCAVEL,COVID-19 BIOLOGIA MOLECULAR,2021-02-28,2021-03-03,3.0,NaT,NaT
721131,742942,2610886,ZORAIDE CARDOSO GRAVENA,F,62,16,4101507,ARAPONGAS,NaT,4101507,ARAPONGAS,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-03-02,2021-03-03,3.0,NaT,NaT
721132,742943,2549118,ZULMIRA MESOMO,F,56,9,4108304,FOZ DO IGUACU,2021-02-22,4108304,FOZ DO IGUACU,COVID-19 BIOLOGIA MOLECULAR,2021-02-26,2021-03-03,3.0,NaT,NaT
721133,742944,2572873,ZULXNEIA LIMA CRESPO ORTIZ,F,39,9,4108304,FOZ DO IGUACU,2021-02-24,4108304,FOZ DO IGUACU,COVID-19 BIOLOGIA MOLECULAR,2021-03-01,2021-03-03,3.0,NaT,NaT


In [10]:
writer = pd.ExcelWriter(join(output,"casos_merged.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_merged.to_excel(writer,sheet_name='casos_merged',index=None)
worksheet = writer.sheets['casos_merged']
auto_fit_columns(worksheet,casos_merged)

obitos_merged.to_excel(writer,sheet_name='obitos_merged',index=None)
worksheet = writer.sheets['obitos_merged']
auto_fit_columns(worksheet,obitos_merged)

writer.save()
writer.close()

KeyboardInterrupt: 

In [ ]:
merge_casos_notifica = pd.merge(merge_casos_notifica, casosn[['id','uf_residencia','classificacao_final','criterio_classificacao','exame','metodo','cod_evolucao','excluir_ficha','manter','duplicado']], on='id', how='left')

merge_casos_notifica.loc[merge_casos_notifica['classificacao_final'].isnull(),'classificacao_final'] = "SEM NOTIFICACAO"
merge_casos_notifica['caso'] = 1
print(len(merge_casos_notifica))

In [ ]:
gcf = merge_casos_notifica.groupby('classificacao_final')[['caso','obito','recuperado','ativo']].sum()
gcf = gcf.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gcf = gcf.append(pd.DataFrame(data=[gcf.sum(axis=0)],columns=gcf.columns,index=['Total']))
gcf

In [ ]:
gce = merge_casos_notifica.groupby('cod_evolucao')[['caso','obito','recuperado','ativo']].sum()
gce = gce.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gce = gce.append(pd.DataFrame(data=[gce.sum(axis=0)],columns=gce.columns,index=['Total']))
gce

In [ ]:
merge_casos_notifica = merge_casos_notifica[['ordem', 'identificacao', 'id','uf_residencia','ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'classificacao_final', 'laboratorio', 'exame', 'metodo', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas','cod_evolucao', 'recuperado', 'caso', 'obito', 'ativo', 'data_obito', 'data_com_obito', 'excluir','excluir_ficha','manter','duplicado']]

In [ ]:
obitos_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['obito']==1) & (merge_casos_notifica['cod_evolucao']!=2)].copy()
obitos_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['obito']==0) & (merge_casos_notifica['cod_evolucao']==2)].copy()
recuperados_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==1) & (merge_casos_notifica['cod_evolucao']!=1)].copy()
recuperados_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==0) & (merge_casos_notifica['cod_evolucao']==1)].copy()

obitos_nao_notifica['diff'] = 'obitos_nao_notifica'
obitos_nao_casos_confirmados['diff'] = 'obitos_nao_casos_confirmados'
recuperados_nao_notifica['diff'] = 'recuperados_nao_notifica'
recuperados_nao_casos_confirmados['diff'] = 'recuperados_nao_casos_confirmados'

diferencas = obitos_nao_notifica.append(obitos_nao_casos_confirmados.append(recuperados_nao_notifica.append(recuperados_nao_casos_confirmados)))

gdiff = diferencas.groupby('diff')[['id']].count() #'caso','obito','recuperado','ativo'
# gdiff = gdiff.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gdiff = gdiff.append(pd.DataFrame(data=[gdiff.sum(axis=0)],columns=gdiff.columns,index=['Total']))
gdiff

In [ ]:
casos_comunicados = merge_casos_notifica

writer = pd.ExcelWriter(join(output,"casos_comunicados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


casos_comunicados.to_excel(writer,sheet_name='casos_comunicados',index=None)
worksheet = writer.sheets["casos_comunicados"]
auto_fit_columns(worksheet,casos_comunicados)

writer.save()
writer.close()

In [ ]:
recuperados = merge_casos_notifica.loc[(merge_casos_notifica['cod_evolucao']==1) & (merge_casos_notifica['obito']!=1)].copy()
recuperados = pd.merge(recuperados,casosn[['id','data_cura_obito','origem']], how='left', on='id')

recuperados['recuperado'] = 'SIM'
recuperados['data'] = date.today()
recuperados['fonte'] = recuperados['origem']
recuperados.loc[recuperados['data_cura_obito'].isnull(),'data_cura_obito'] = date.today()

recuperados = recuperados[['ordem','identificacao','id','recuperado','data','fonte','mun_resid','ibge_resid','paciente','sexo','idade','data_cura_obito']]

writer = pd.ExcelWriter(join(output,"recuperados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


recuperados.to_excel(writer,sheet_name='recuperados',index=None)
worksheet = writer.sheets["recuperados"]
auto_fit_columns(worksheet,recuperados)

writer.save()
writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_classificacao_final.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gcf.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gcf)

for group, df in merge_casos_notifica.groupby('classificacao_final'):
    group = group[:30] if len(group) > 30 else group
    df.to_excel(writer,sheet_name=group,index=None)
    worksheet = writer.sheets[group]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_evolucao.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gce.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gce)

for cod_evolucao, df in merge_casos_notifica.groupby('cod_evolucao'):
    df.to_excel(writer,sheet_name=f"{cod_evolucao}",index=None)
    worksheet = writer.sheets[f"{cod_evolucao}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_diferencas.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gdiff.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gdiff)

for diff, df in diferencas.groupby('diff'):
    diff = diff[:30] if len(diff) > 30 else diff
    df.to_excel(writer,sheet_name=f"{diff}",index=None)
    worksheet = writer.sheets[f"{diff}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()